In [13]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.debugging.experimental.disable_dump_debug_info
#-----------------------------------------------------------------------------------------#
from keras import Sequential
from keras import layers
from tensorflow import keras
from sklearn import metrics
import numpy as np
import argparse
from scipy.io import arff
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import random

def parse_arguments():
    parser = argparse.ArgumentParser(description="FeGAN OD")
    parser.add_argument("--path", default="../Resources/Datasets/Arrhythmia_withoutdupl_norm_02_v01.arff",
                        help="Data path")
    parser.add_argument("--lr_gen", type=float, default=0.01, help="Learning rate generator")
    parser.add_argument("--lr_dis", type=float, default=0.01, help="Learning rate discriminator")
    parser.add_argument("--stop_epochs", type=int, default = 5, help="Generator stops training after stop_epochs")
    parser.add_argument("--k", type=int, default=10, help="Number of discriminators")
    
    return parser.parse_args()

'''
    Calculate AUC and print it
'''
def AUC(truth, decision):
    output = metrics.roc_auc_score(truth, decision)
    print("AUC: " + str(output))
    return output

def set_seed(seed):
    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)
    tf.config.experimental.enable_op_determinism()
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    keras.utils.set_random_seed(seed) #seeds numpy, random and tf all at once
    os.environ["PYTHONHASSEED"] = str(seed)

#-----------------------------------------------------------------------------------------#


'''
    Create the generator. Uses two dense layers and relu activatoin
'''
def create_gen():
    generator = Sequential()
    generator.add(layers.Dense(latent_size, input_dim=latent_size, activation="relu", kernel_initializer=keras.initializers.Identity(gain=1.0)))
    generator.add(layers.Dense(latent_size, activation='relu', kernel_initializer=keras.initializers.Identity(gain=1.0)))
    latent = keras.Input(shape=(latent_size,))
    fake_data = generator(latent)
    return keras.Model(latent, fake_data)

'''
    Create the discriminator. USes two dense layers and relu activation
'''
def create_dis(sub_size):
    discriminator = Sequential()
    discriminator.add(layers.Dense(np.ceil(np.sqrt(data_size)), input_dim=sub_size, activation='relu', kernel_initializer= keras.initializers.VarianceScaling(scale=1.0, mode='fan_in', distribution='normal', seed=None)))
    discriminator.add(layers.Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.VarianceScaling(scale=1.0, mode='fan_in', distribution='normal', seed=None)))
    data = keras.Input(shape=(sub_size,))
    fake = discriminator(data)
    return keras.Model(data, fake)

def load_data():
    arff_data = arff.loadarff(r"C:\Users\denis\Documents\BA\FSS_GAN\Resources\Datasets\Arrhythmia_withoutdupl_norm_02_v01.arff")
    df = pd.DataFrame(arff_data[0])
    df["outlier"] = pd.factorize(df["outlier"], sort=True)[0] #maybe flip
    data_x = df.iloc[:,:-2]
    data_y = df.iloc[:,-1]
    
    return data_x, data_y

'''
    Plot the loss of the models. Generator in blue.
'''
def plot(train_history):
    dy = train_history['discriminator_loss']
    gy = train_history['generator_loss']
    auc_y = train_history['auc']
    for i in range(k):
        names['dy_' + str(i)] = train_history['sub_discriminator{}_loss'.format(i)]
    x = np.linspace(1, len(gy), len(gy))
    fig, ax = plt.subplots()
    ax.plot(x, gy, color='blue')
    ax.plot(x, dy,color='red')
    ax.plot(x, auc_y, color='yellow', linewidth = '3')
    for i in range(k):
        ax.plot(x, names['dy_' + str(i)], color='green', linewidth='0.5')
    plt.show()
    
'''
    Randomly draw subspaces for each sub_discriminator. Store them in names[]
'''
def draw_subspaces(dimension, k):
    dims = random.sample(range(dimension), k)
    for i in range(k):
        names["subspaces"+str(i)] = random.sample(range(dimension), dims[i])
    

In [17]:
set_seed(777)
train = True
    
#args = parse_arguments()
data_x, data_y = load_data()
data_size = data_x.shape[0] # n := number of samples
latent_size = data_x.shape[1] # dimension of the data set

k = 10
train_history = defaultdict(list)
names = locals()
epochs = 5 * 3
stop = 0

        
generator = create_gen()
generator.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01), loss='binary_crossentropy')
latent = keras.Input(shape=(latent_size,))
create = 0 # used to initialize sum of the sub_discriminators
        
draw_subspaces(latent_size,k)

In [24]:
for i in range(k):
    names["sub_discriminator" + str(i)] = create_dis(len(names["subspaces"+str(i)]))
    names["fake" + str(i)] = generator(latent) # generate the fake data of the generator
            #names["sub_discriminator" + str(i)].trainable = False

    print(names["fake" + str(i)])
    #print(names["subspaces"+str(i)])
    #print(tf.gather(names["fake" + str(i)],[names["subspaces"+str(i)]])) # Problem: choosing relevant subspaces doesnt work like that
    names["fake" + str(i)] = names["sub_discriminator" + str(i)](names["fake" + str(i)][:,names["subspaces"+str(i)]]) # D(G(z))
    names["sub_discriminator_sum"] = np.zeros(latent_size) # das ist evtl falsch
    for i in range(k):
        for j in range(len(names["subspaces"+str(i)])):
            names["sub_discriminator_sum"][names["subspaces"+str(i)][j]] += names["fake"+str(i)][j]
            
    names["sub_discriminator" + str(i)].compile(optimizer=keras.optimizers.SGD(learning_rate=args.lr_dis), loss='binary_crossentropy')
            
names["sub_discriminator_sum"] /= k
names["combine_model"] = keras.Model(latent, names["sub_discriminator_sum"]) # model with the average decision. Used to train the generator.
names["combine_model"].compile(optimizer=keras.optimizers.SGD(learning_rate=args.lr_gen), loss='binary_crossentropy')
        

KerasTensor(type_spec=TensorSpec(shape=(None, 259), dtype=tf.float32, name=None), name='model_1/sequential_1/dense_3/Relu:0', description="created by layer 'model_1'")


TypeError: Exception encountered when calling layer "tf.__operators__.getitem_1" (type SlicingOpLambda).

Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got [39, 170, 53, 0, 214, 164, 147, 86, 73, 149, 68, 27, 161, 257, 222, 256, 79, 128, 32, 208, 207, 210, 106, 150, 205, 8, 89, 156, 37, 252, 134, 19, 139, 226, 169, 70, 244, 198, 176, 127, 24, 140, 115, 43, 2, 112, 65, 55, 168, 144, 98, 22, 96, 45, 122, 190, 16, 137, 100, 143, 81, 78, 48, 218, 44, 31, 141, 249, 233, 1, 120, 235, 67, 165, 248, 254, 215, 178, 30, 253, 219, 239, 193, 167, 224, 181, 103, 108, 102, 113, 87, 187, 163, 160, 217, 91, 138, 154, 129, 13, 231, 159, 95, 250, 148, 130, 111, 38, 155, 125, 101, 3, 71, 191, 166, 132, 204]

Call arguments received by layer "tf.__operators__.getitem_1" (type SlicingOpLambda):
  • tensor=tf.Tensor(shape=(None, 259), dtype=float32)
  • slice_spec=({'start': 'None', 'stop': 'None', 'step': 'None'}, ['39', '170', '5